In [2]:
# import
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
from tqdm import tqdm_notebook

from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import requests
import urllib.parse

## google검색으로 IMDb URL가져오기(하루 500개 한정) 

In [3]:
# 엑셀 파일을 읽어옵니다.
just = pd.read_excel('../data/justwatch_2.xlsx', index_col=0)
just.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country
0,글리치,NaN,2022,1,54min,"드라마, 코미디, 스릴러, SF",15,대한민국
1,보라! 데보라,NaN,2023,1,1시간 8분,"코미디, 드라마",15,대한민국
2,산토,Santo,2022,1,45min,"범죄, 드라마, 스릴러, 액션",NaN,스페인
3,프로이트의 살인 해석,Freud,2020,1,53min,"드라마, 스릴러, 범죄",19,"독일, 오스트리아"
4,콜래트럴 이펙트,Collateral,2018,1,57min,"범죄, 드라마, 스릴러",NaN,영국


In [4]:
def get_imdb_result(query):
    # NaN 값인 경우 처리합니다.
    if pd.isna(query):
        return None
    
    # 검색 쿼리를 URL 인코딩합니다.
    query = urllib.parse.quote(query + " TV IMDb")
    
    # 구글 검색 URL을 구성합니다.
    url = f"https://www.google.com/search?q={query}"
    
    # 요청 헤더를 설정합니다.
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    
    # 구글 검색 페이지에 요청을 보냅니다.
    response = requests.get(url, headers=headers)
    
    # 응답의 상태 코드를 확인합니다.
    if response.status_code == 200:
        # HTML 내용을 파싱합니다.
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 모든 검색 결과를 가져옵니다.
        search_results = soup.find_all("div", class_="g")
        
        # IMDb 사이트의 URL을 포함한 첫 번째 검색 결과를 찾습니다.
        for result in search_results:
            link = result.find("a")
            if link and 'href' in link.attrs:
                url = link['href']
                # IMDb 사이트의 URL인지 확인합니다.
                if "imdb.com" in url:
                    title = result.find("h3").text
                    return {"title": title, "url": url}
    
    return None

In [5]:
# 기존 df에 다른 컬럼에 넣기
def get_imdb_results(df):
    imdb_titles = []
    imdb_urls = []
    for i, row in df.iterrows():
        query = str(row["original_title"]) + str(row["title"]) + str(row["year"])  # NaN 값이 있을 수 있으므로 문자열로 변환합니다.
        result = get_imdb_result(query)
        if result:
            imdb_titles.append(result["title"])
            imdb_urls.append(result["url"])
        else:
            imdb_titles.append(None)
            imdb_urls.append(None)
    return imdb_titles, imdb_urls

# IMDb 제목과 URL을 가져와서 just 데이터프레임에 새로운 열로 추가합니다.
just["IMDb_title"], just["IMDb_URL"] = get_imdb_results(just)

In [6]:
just.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL
0,글리치,NaN,2022,1,54min,"드라마, 코미디, 스릴러, SF",15,대한민국,Glitch (TV Series 2022,https://www.imdb.com/title/tt14735382/
1,보라! 데보라,NaN,2023,1,1시간 8분,"코미디, 드라마",15,대한민국,True to Love (TV Series 2023),https://www.imdb.com/title/tt26763160/
2,산토,Santo,2022,1,45min,"범죄, 드라마, 스릴러, 액션",NaN,스페인,Santo (TV Series 2022,https://www.imdb.com/title/tt14278524/
3,프로이트의 살인 해석,Freud,2020,1,53min,"드라마, 스릴러, 범죄",19,"독일, 오스트리아",Freud (TV Series 2020),https://www.imdb.com/title/tt8667956/
4,콜래트럴 이펙트,Collateral,2018,1,57min,"범죄, 드라마, 스릴러",NaN,영국,Collateral (TV Mini Series 2018),https://www.imdb.com/title/tt6729080/


In [5]:
# 저장
just.to_csv('../data/justwatch_url.csv', sep="/", encoding="utf-8")

just.to_excel('../data/justwatch_url.xlsx')

---

### 결과를 또다른 데이터프레임에 넣을 경우

In [7]:
# 데이터프레임에 있는 각 행의 내용에 대해 IMDb 검색 결과를 가져옵니다.
def get_imdb_results_df(df):
    imdb_results = []
    for i, row in df.iterrows():
        query = str(row["original_title"]) + str(row["title"]) + str(row["year"])  # NaN 값이 있을 수 있으므로 문자열로 변환합니다.
        result = get_imdb_result(query)
        imdb_results.append(result)
    return imdb_results

# 데이터프레임의 처음 5개 행에 대한 IMDb 검색 결과를 가져옵니다.
imdb_results_df = get_imdb_results_df(just)

# 결과를 출력합니다.
for result in imdb_results_df:
    if result:
        print("제목:", result["title"])
        print("URL:", result["url"])
    else:
        print("검색 결과를 찾을 수 없습니다.")

제목: Glitch (TV Series 2022
URL: https://www.imdb.com/title/tt14735382/
제목: True to Love (TV Series 2023)
URL: https://www.imdb.com/title/tt26763160/
제목: Santo (TV Series 2022
URL: https://www.imdb.com/title/tt14278524/
제목: Freud (TV Series 2020)
URL: https://www.imdb.com/title/tt8667956/
제목: Collateral (TV Mini Series 2018)
URL: https://www.imdb.com/title/tt6729080/
제목: The Trial (TV Series 2019
URL: https://www.imdb.com/title/tt10549212/
제목: Tore (2023) - TV Series
URL: https://www.imdb.com/title/tt24132876/
제목: Mismatched (TV Series 2020
URL: https://www.imdb.com/title/tt12719250/
제목: Insider (TV Series 2022)
URL: https://www.imdb.com/title/tt14731504/
제목: Wanderlust (TV Series 2018)
URL: https://www.imdb.com/title/tt7608238/
제목: Twist of Fate (TV Series 2021)
URL: https://www.imdb.com/title/tt14411816/
제목: The Forest of Love: Deep Cut (TV Mini Series 2020)
URL: https://www.imdb.com/title/tt12079236/releaseinfo/
제목: Heirs to the Land (TV Series 2022)
URL: https://www.imdb.com/title/t

## 비영어권에서는 오류가 많은 점 주의